# Making Julia as Fast as C++

In [1]:
using CPUTime
using TimerOutputs
using LinearAlgebra
using BenchmarkTools
using Printf

macro bash_str(s) open(`bash`,"w",stdout) do io; print(io, s); end; end

@bash_str (macro with 1 method)

The myth says that Julia can achieve the same computing performance than a compiled language like C++ and FORTRAN. I've spent the last few days trying to optimize a piece of my code that was a hardcore bottle neck, and here I'm summarizing my conclusions and tips for speeding Julia up.

# Achieving C++ Performance in Julia

## Problem Definition

In the vortex particle method we are interested in calculating the velocity and Jacobian of the velocity that a field of N particles induces at an arbitrary position $\mathbf{x}$, calculated as

\begin{align}
    \bullet \quad &
		{\bf u}\left( {\bf x} \right) = \sum\limits_p^N g_\sigma\left( {\bf x}-{\bf x}_p \right)
						{\bf K}\left( {\bf x}-{\bf x}_p \right)   \times
			            \boldsymbol\Gamma_p 
    \\
    \bullet \quad &
        \frac{\partial {\bf u}}{\partial x_j}\left( {\bf x} \right) 
        = \sum\limits_p^N \left[
            \left(
                \frac{1}{\sigma }
                 \frac{\Delta x_j}{\Vert \Delta \mathbf{x} \Vert}
                 \frac{\partial g}{\partial r}
                 \left( 
                             \frac{\Vert \Delta\mathbf{x} \Vert}{\sigma} 
                 \right) -
                 3 g_\sigma\left( \Delta{\bf x} \right) 
                 \frac{\Delta x_j}{\Vert \Delta\mathbf{x} \Vert^2} 
            \right)
            {\bf K}\left( \Delta\mathbf{x} \right)  \times \boldsymbol\Gamma_p -
            \frac{g_\sigma\left( \Delta{\bf x} \right) }{4\pi \Vert \Delta{\bf x} \Vert^3}
            \delta_{ij} \times \boldsymbol\Gamma_p
        \right]
,\end{align}

with ${\bf K}$ the singular Newtonian kernel ${\bf K}\left( {\bf x}\right)=-\frac{{\bf x}}{4\pi \Vert{\bf x}\Vert^3}$, $g_\sigma$ a regularizing function of smoothing radius $\sigma$ as $g_\sigma(\mathbf{x}) = g\left( \frac{\Vert \mathbf{x} \Vert}{\sigma} \right)$, and $\mathbf{x}_p$ and $\boldsymbol{\Gamma}_p$ the position and vectorial strength of the $p$-th particle, respectively. Furthermore, the governing equations of the method require evaluating the velocity $\mathbf{u}$ and its Jacobian $\mathbf{J}$ that the collection of particles induces at the position of every individual particle, leading to a computational complexity $\mathcal{O}(N^2)$.

Choosing Winckelmans' regularizing kernel 
\begin{align}
    \bullet \quad &
        g(r) = r^3 \frac{r^2 + 5/2}{\left( r^2 + 1 \right)^{5/2}}
    \\
    \bullet \quad &
        \frac{\partial g}{\partial r} (r) = \frac{15}{2} 
            \frac{r^2}{\left( r^2 + 1 \right)^{7/2}}
,\end{align}
the above equations are implemented in C++ as follows:

```c++
void P2P(Cell * Ci, Cell * Cj) {
  real_t const4 = 1/(4*M_PI);
  Particle * Pi = Ci->particle;
  Particle * Pj = Cj->particle;

  for (int i=0; i<Ci->numParticles; i++) {
    vec3 U = 0;
    vec9 J = 0;
    
    for (int j=0; j<Cj->numParticles; j++) {
      vec3 dX = Pi[i].X - Pj[j].X;
      real_t r = std::pow(dX[0]*dX[0] + dX[1]*dX[1] + dX[2]*dX[2], 0.5) + EPS;
      real_t ros = r/Pj[j].sigma;

      // u(x) = ∑g_σ(x−xp) K(x−xp) × Γp
      real_t aux0 = std::pow(ros*ros + 1.0, 2.5);
      real_t g_sgm = ros*ros*ros * (ros*ros + 2.5) / aux0;
      real_t dgdr = 7.5 * ros*ros / ( aux0 * (ros*ros + 1.0) );
      real_t aux = (- const4 / r*r*r) * g_sgm;
      U[0] += ( dX[1]*Pj[j].q[2] - dX[2]*Pj[j].q[1] ) * aux;
      U[1] += ( dX[2]*Pj[j].q[0] - dX[0]*Pj[j].q[2] ) * aux;
      U[2] += ( dX[0]*Pj[j].q[1] - dX[1]*Pj[j].q[0] ) * aux;

      // ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = (- const4 / r*r*r) * (dgdr/(Pj[j].sigma*r) - 3.0*g_sgm/r*r);
      for (int k=0; k<3; k++){
        J[0 + k] += ( dX[1]*Pj[j].q[2] - dX[2]*Pj[j].q[1] )* aux*dX[k];
        J[3 + k] += ( dX[2]*Pj[j].q[0] - dX[0]*Pj[j].q[2] )* aux*dX[k];
        J[6 + k] += ( dX[0]*Pj[j].q[1] - dX[1]*Pj[j].q[0] )* aux*dX[k];
      }

      // Adds the Kronecker delta term
      aux = - const4 * g_sgm / r*r*r;
      // k=1
      J[3 + 0] -= aux * Pj[j].q[2];
      J[6 + 0] += aux * Pj[j].q[1];
      // k=2
      J[0 + 1] += aux * Pj[j].q[2];
      J[6 + 1] -= aux * Pj[j].q[0];
      // k=3
      J[0 + 2] -= aux * Pj[j].q[1];
      J[3 + 2] += aux * Pj[j].q[0];
    }

    Pi[i].p += U;
    Pi[i].J += J;
  }
}
```

Ten function calls of  this `P2P` function on 125 particles in **C++ results in an average wall-clock time of 6 ms (0.6 ms per P2P call)** on my Dell Latitude 5580 laptop (Intel® Core™ i7-7820HQ CPU @ 2.90GHz × 8 ) in only one process:

In [42]:
bash"""
./code/P2Pcxx
"""

10 P2P function calls on 125 particles
	CPU time used: 0.0010000000 ms
	Wall clock time passed: 5.8687950000 ms

	---------------------------------------------------------
	-	elapsed Wall clock time: 0.0058687950 seconds
	---------------------------------------------------------


## Julia Setup

In order to proceed to code this in Julia, first we define a Particle type to store all the information. Here are two versions of the Particle type, one that we will see is very inneficient and the other one that achieves better computational efficiency by explicitly declaring its properties as concrete types:

In [3]:
"""
This is a particle type with properties of ambiguous types
"""
struct ParticleAmbiguous

  # User inputs
  X                                       # Position
  Gamma                                   # Vectorial circulation
  sigma                                   # Smoothing radius

  # Properties
  U                                       # Velocity at particle
  J                                       # Jacobian at particle J[i,j]=dUi/dxj


  ParticleAmbiguous(
            X, Gamma, sigma;
            U=zeros(3), J=zeros(3,3)
           ) = new(
             X, Gamma, sigma,
             U, J
           )
end

"""
This is a particle type with properties that are specified
as concrete types
"""
struct ParticleConcrete

  # User inputs
  X::Array{Float64, 1}                      # Position
  Gamma::Array{Float64, 1}                  # Vectorial circulation
  sigma::Float64                            # Smoothing radius

  # Properties
  U::Array{Float64, 1}                      # Velocity at particle
  J::Array{Float64, 2}                      # Jacobian at particle J[i,j]=dUi/dxj


  ParticleConcrete(
            X, Gamma, sigma;
            U=zeros(Float64, 3), J=zeros(Float64, 3,3)
           ) = new(
             X, Gamma, sigma,
             U, J
           )
end

ParticleConcrete

This is a function that creates and adds particles to an array in a box lattice pattern:

In [4]:
"Adds particles in a regular box lattice"
function add_lattice!(particles; nx=5, ny=5, nz=5, lx=1, ly=1, lz=1, 
                                  C=zeros(3),
                                  lambda=1.5, Gamma=ones(3))
  # Parameters
  # nx, ny, nz = 5, 5, 5          # Particles in each dimension
  # lx, ly, lz = 1, 1, 1          # Length of box in each dimension
  # C = zeros(3)                  # Box center
  # lambda = 1.5                  # Particle overlap in x-direction

  # Adds particles in a regular lattice
  xs = range(0, stop=lx, length=nx) .- lx/2 .+ C[1]
  ys = range(0, stop=ly, length=ny) .- ly/2 .+ C[2]
  zs = range(0, stop=lz, length=nz) .- lz/2 .+ C[3]
  for k in 1:nz
    for j in 1:ny
      for i in 1:nx
        X = [xs[i], ys[j], zs[k]]
        sigma = lx/nx*lambda
        push!(particles, eltype(particles)(X, Gamma, sigma))
      end
    end
  end
end

add_lattice!

Here we create arrays of 125 particles arranged in a 5x5x5 box:

In [5]:
n = 5

particlesA = ParticleAmbiguous[]
add_lattice!(particlesA; nx=n, ny=n, nz=n)

particlesC = ParticleConcrete[]
add_lattice!(particlesC; nx=n, ny=n, nz=n)

and here are some constant values we will reuse later on:

In [6]:
const const4 = 1/(4*pi)
const EPS = 1e-14;

Here we will store our benchmark results and we define a function for comparing times:

In [7]:
benchtime = Dict{Union{Function, String}, Float64}( "C++" => 5.844/10);

global compfun = nothing
global compfunargs = nothing

function compare(fun, ref::Union{Function, String}, funargs; 
                    verbose=true, reverse=false)
    
    global compfun = fun
    global compfunargs = funargs
    
    benchtime[fun] = (@belapsed compfun(compfunargs...))*1000
    
    if verbose
        printcomparison(fun, ref, reverse)
    end
end

function printcomparison(fun, ref, reverse)
    ratio = benchtime[fun]/benchtime[ref]
    if reverse
        @printf "%-20s is %5.2f times faster than %20s (%5.3fms vs %5.3fms)\n" fun 1/ratio ref benchtime[fun] benchtime[ref]
    else
        @printf "%-20s is %5.2f times faster than %20s (%5.3fms vs %5.3fms)\n" ref ratio fun benchtime[ref] benchtime[fun]
    end
end

printcomparison (generic function with 1 method)

## Fix #1: Concrete types

Our first inclination in Julia is to make the code as general and easy to understand as possible. Here is the most general form of the code where no types are specified and the kernel is unspecified:

In [8]:
g_wnklmns(r) = r^3 * (r^2 + 2.5) / (r^2 + 1)^2.5
dgdr_wnklmns(r) = 7.5 * r^2 / (r^2 + 1)^3.5

function P2P_general(sources::Array{ParticleAmbiguous},
                     targets::Array{ParticleAmbiguous}, 
                     g::Function, dgdr::Function)

  for Pi in targets 
    for Pj in sources    
            
      dX = Pi.X - Pj.X
      r = norm(dX) + EPS
            
      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)  
            
      # K × Γp
      crss = cross(-const4 * dX / r^3, Pj.Gamma) 

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[:] += gsgm * crss

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss -
                                  gsgm * const4/r^3 * 
                                  cross([i==j for i in 1:3], Pj.Gamma) )
      end

    end
  end
end

P2P_general (generic function with 1 method)

In [9]:
this_fun = P2P_general
args = (particlesA, particlesA, g_wnklmns, dgdr_wnklmns)

compare(this_fun, "C++", args)

@benchmark this_fun(args...)

C++                  is 115.16 times faster than          P2P_general (0.584ms vs 67.297ms)


BenchmarkTools.Trial: 
  memory estimate:  69.86 MiB
  allocs estimate:  1437500
  --------------
  minimum time:     66.281 ms (4.12% GC)
  median time:      69.037 ms (4.52% GC)
  mean time:        71.489 ms (5.60% GC)
  maximum time:     126.785 ms (37.55% GC)
  --------------
  samples:          70
  evals/sample:     1

Here we see that C++ is over 100 times faster than our Julia code. The problem is that **without foreknowledge of the types to be handled in this function, Julia can't optimize the function during compilation**. Here we modify the function to explicitly state the types of all internal variables:

In [10]:
function P2P_explicittypes(sources::Array{ParticleAmbiguous},
                             targets::Array{ParticleAmbiguous}, 
                             g::Function, dgdr::Function)

  for Pi in targets 
    for Pj in sources    
            
      dX::Array{Float64, 1} = Pi.X - Pj.X
      r::Float64 = norm(dX) + EPS
            
      # Regularizing function and deriv
      gsgm::Float64 = g(r/Pj.sigma)
      dgsgmdr::Float64 = dgdr(r/Pj.sigma)  
            
      # K × Γp
      crss::Array{Float64, 1} = cross(-const4 * dX / r^3, Pj.Gamma) 

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[:] += gsgm * crss

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss -
                                  gsgm * const4/r^3 * 
                                  cross([i==j for i in 1:3], Pj.Gamma) )
      end

    end
  end
end

P2P_explicittypes (generic function with 1 method)

In [11]:
prev_fun = this_fun
this_fun = P2P_explicittypes
args = (particlesA, particlesA, g_wnklmns, dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_explicittypes    is  1.12 times faster than          P2P_general (60.077ms vs 67.297ms)
C++                  is 102.80 times faster than    P2P_explicittypes (0.584ms vs 60.077ms)


BenchmarkTools.Trial: 
  memory estimate:  66.52 MiB
  allocs estimate:  1218750
  --------------
  minimum time:     59.109 ms (5.25% GC)
  median time:      61.635 ms (5.35% GC)
  mean time:        62.209 ms (6.28% GC)
  maximum time:     112.506 ms (42.41% GC)
  --------------
  samples:          81
  evals/sample:     1

Specifying types of internal variables speed the computation a modest amount (1.1x), but is not enough. Julia has a built-in macro `@code_warntype` that helps identify ambiguous types that keeps the compiler from optimizing a section of code.

In [12]:
? @code_warntype

```
@code_warntype
```

Evaluates the arguments to the function or macro call, determines their types, and calls [`code_warntype`](@ref) on the resulting expression.


In [13]:
? code_warntype

search: code_warntype @code_warntype



```
code_warntype([io::IO], f, types; verbose_linetable=false)
```

Prints lowered and type-inferred ASTs for the methods matching the given generic function and type signature to `io` which defaults to `stdout`. The ASTs are annotated in such a way as to cause "non-leaf" types to be emphasized (if color is available, displayed in red). This serves as a warning of potential type instability. Not all non-leaf types are particularly problematic for performance, so the results need to be used judiciously. In particular, unions containing either [`missing`](@ref) or [`nothing`](@ref) are displayed in yellow, since these are often intentional. If the `verbose_linetable` keyword is set, the linetable will be printed in verbose mode, showing all available information (rather than applying the usual heuristics). See [`@code_warntype`](@ref man-code-warntype) for more information.


Here is what the macro tells us about our function (reading the output is an art that you learn the more you try it, the important thing is to know that red means no good):

In [14]:
@code_warntype P2P_explicittypes(args...)

Body::Nothing
│╻╷╷  iterate5  1 ── %1   = (Base.arraylen)(targets)::Int64
││╻╷   iterate   │    %2   = (Base.sle_int)(0, %1)::Bool
│││╻    <   │    %3   = (Base.bitcast)(UInt64, %1)::UInt64
││││╻    <   │    %4   = (Base.ult_int)(0x0000000000000000, %3)::Bool
││││╻    &   │    %5   = (Base.and_int)(%2, %4)::Bool
│││     └───        goto #3 if not %5
│││╻    getindex   2 ── %7   = (Base.arrayref)(false, targets, 1)::ParticleAmbiguous
│││     └───        goto #4
│││     3 ──        goto #4
││      4 ┄─ %10  = φ (#2 => false, #3 => true)::Bool
││      │    %11  = φ (#2 => %7)::ParticleAmbiguous
││      │    %12  = φ (#2 => 2)::Int64
││      └───        goto #5
│       5 ── %14  = (Base.not_int)(%10)::Bool
│       └───        goto #36 if not %14
│       6 ┄─ %16  = φ (#5 => %11, #35 => %175)::ParticleAmbiguous
│       │    %17  = φ (#5 => %12, #35 => %176)::Int64
│╻╷╷  iterate6  │    %18  = (Base.arraylen)(sources)::Int64
││╻╷   iterate   │    %19  = (Base.sle_int)(0, %18)::Bool
│││╻    < 

││╻╷   *   │    %109 = (Base.mul_float)(%58, 3.0)::Float64
││╻    *   │    %110 = (Base.mul_float)(%109, %108)::Float64
││╻    *   │    %111 = (Base.mul_float)(%52, %52)::Float64
│╻    /   │    %112 = (Base.div_float)(%110, %111)::Float64
│╻    *   │    %113 = invoke Base.broadcast(Base.:*::typeof(*), %112::Float64, %81::Array{Float64,1})::Array{Float64,1}
│       │    %114 = (%107 - %113)::Any
│       │    %115 = Main.const4::Core.Compiler.Const(0.07957747154594767, false)
│╻    *   │    %116 = (Base.mul_float)(%58, %115)::Float64
││╻╷   *   │    %117 = (Base.mul_float)(%52, %52)::Float64
│││┃    *   │    %118 = (Base.mul_float)(%117, %52)::Float64
│╻    /   │    %119 = (Base.div_float)(%116, %118)::Float64
│       │    %120 = %new(getfield(Main, Symbol("##15#16")){Int64}, %98)::getfield(Main, Symbol("##15#16")){Int64}
││╻╷   Type   │           (Base.ifelse)(true, 3, 0)
│││     │    %122 = %new(UnitRange{Int64}, 1, 3)::UnitRange{Int64}
││╻    Type   │    %123 = %new(Base.Generator{Uni

Here are the things to notice from what `@code_warntype` is telling us:

* Any operation over a particle property (`X`, `Gamma`, `sigma`) is red, meaning that the compiler couldn't identify a concrete type for them.


* The output of both functions `g` and `dgdr` are type-indetermined.


* The output of the LinearAlgebra function `cross` is type-indetermined.


* The external variables where the function store final calculations (`U` and `J`) are type-indetermined.

In order to start fixing these type-related issues, first we start by using a well-defined struct for storing information. In the previous function we were using the `ParticleAmbiguous` struct that doesn't declare any types for its properties, raising most of the type-indifined issues. We solve this by replacing the ambiguous struct with the `ParticleConcrete` struct that specifies all its properties as concrete types. It is important to mention that **declaring types as abstract types (*e.g.,* `X::Real` instead of `X::Float64`) still keeps compiler from being able to optimize the code (this is not an issue in function declarations, since JIT creates a function of the specific concrete type the function has been called upon on the fly)**.

In [15]:
function P2P_concretestruct(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete}, 
                             g::Function, dgdr::Function)

  for Pi in targets 
    for Pj in sources    
            
      dX = Pi.X - Pj.X
      r = norm(dX) + EPS
            
      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)
            
      # K × Γp
      crss = cross(-const4 * dX / r^3, Pj.Gamma) 

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[:] += gsgm * crss

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss -
                                  gsgm * const4/r^3 * 
                                  cross([i==j for i in 1:3], Pj.Gamma) )
      end

    end
  end
end

P2P_concretestruct (generic function with 1 method)

In [16]:
prev_fun = this_fun
this_fun = P2P_concretestruct
args = (particlesC, particlesC, g_wnklmns, dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_concretestruct   is  3.07 times faster than    P2P_explicittypes (19.565ms vs 60.077ms)
C++                  is 33.48 times faster than   P2P_concretestruct (0.584ms vs 19.565ms)


BenchmarkTools.Trial: 
  memory estimate:  58.89 MiB
  allocs estimate:  718750
  --------------
  minimum time:     19.678 ms (10.43% GC)
  median time:      21.080 ms (15.06% GC)
  mean time:        21.390 ms (15.04% GC)
  maximum time:     71.203 ms (64.59% GC)
  --------------
  samples:          234
  evals/sample:     1

Voilà! By **using a concrete struct we immediately gain a 3x speed up**, while avoiding explicitly specifying (*i.e.,* hard coding) the types of our internal variables.

Running the type analysis again we see that all our type-indetermined issues have now been solved:

In [17]:
@code_warntype P2P_concretestruct(args...)

Body::Nothing
│╻╷╷       iterate5  1 ── %1   = (Base.arraylen)(targets)::Int64
││╻╷        iterate   │    %2   = (Base.sle_int)(0, %1)::Bool
│││╻         <   │    %3   = (Base.bitcast)(UInt64, %1)::UInt64
││││╻         <   │    %4   = (Base.ult_int)(0x0000000000000000, %3)::Bool
││││╻         &   │    %5   = (Base.and_int)(%2, %4)::Bool
│││          └───        goto #3 if not %5
│││╻         getindex   2 ── %7   = (Base.arrayref)(false, targets, 1)::ParticleConcrete
│││          └───        goto #4
│││          3 ──        goto #4
││           4 ┄─ %10  = φ (#2 => false, #3 => true)::Bool
││           │    %11  = φ (#2 => %7)::ParticleConcrete
││           │    %12  = φ (#2 => 2)::Int64
││           └───        goto #5
│            5 ── %14  = (Base.not_int)(%10)::Bool
│            └───        goto #89 if not %14
│            6 ┄─ %16  = φ (#5 => %11, #88 => %359)::ParticleConcrete
│            │    %17  = φ (#5 => %12, #88 => %360)::Int64
│╻╷╷       iterate6  │    %18  = (Base.arrayle

││           │    %115 = invoke Base.vect(%108::Float64, %111::Vararg{Float64,N} where N, %114)::Array{Float64,1}
││           └───        goto #25
│╻         getproperty19 25 ─ %117 = (Base.getfield)(%16, :U)::Array{Float64,1}
││╻         length   │    %118 = (Base.arraylen)(%117)::Int64
││╻╷        similar   │    %119 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Float64,1}, svec(Any, Int64), :(:ccall), 2, Array{Float64,1}, :(%118), :(%118)))::Array{Float64,1}
││╻╷        >   │    %120 = (Base.slt_int)(0, %118)::Bool
││           └───        goto #27 if not %120
││           26 ─        invoke Base.unsafe_copyto!(%119::Array{Float64,1}, 1::Int64, %117::Array{Float64,1}, 1::Int64, %118::Int64)
││           27 ─        goto #28
│╻         *   28 ─ %124 = invoke Base.broadcast(Base.:*::typeof(*), %62::Float64, %115::Array{Float64,1})::Array{Float64,1}
│            │    %125 = (Core.tuple)(%124)::Tuple{Array{Float64,1}}
│╻         +   └───        goto #34 if not true
││           2

││╻         *   │    %207 = invoke Base.broadcast(Base.:*::typeof(*), %206::Float64, %115::Array{Float64,1})::Array{Float64,1}
│╻         getindex   │    %208 = (Base.arrayref)(true, %38, %170)::Float64
││╻╷        *   │    %209 = (Base.mul_float)(%62, 3.0)::Float64
││╻         *   │    %210 = (Base.mul_float)(%209, %208)::Float64
││╻         *   │    %211 = (Base.mul_float)(%42, %42)::Float64
│╻         /   │    %212 = (Base.div_float)(%210, %211)::Float64
│╻         *   │    %213 = invoke Base.broadcast(Base.:*::typeof(*), %212::Float64, %115::Array{Float64,1})::Array{Float64,1}
│╻         -   │           invoke Base.promote_shape(%207::Array{Float64,1}, %213::Array{Float64,1})
││           │    %215 = invoke Base.broadcast(Base.:-::typeof(-), %207::Array{Float64,1}, %213::Array{Float64,1})::Array{Float64,1}
│            │    %216 = Main.const4::Core.Compiler.Const(0.07957747154594767, false)
│╻         *   │    %217 = (Base.mul_float)(%62, %216)::Float64
││╻╷        *   │    %218 = 

││││╻╷╷       uncolon   │    %294 = %new(Base.Slice{Base.OneTo{Int64}}, %291)::Base.Slice{Base.OneTo{Int64}}
││╻         _setindex!   └───        goto #72 if not true
│││          68 ─ %296 = (Core.tuple)(%294, %170)::Tuple{Base.Slice{Base.OneTo{Int64}},Int64}
│││╻╷╷       checkbounds   │    %297 = (Base.arraysize)(%286, 1)::Int64
││││┃││       checkbounds   │    %298 = (Base.arraysize)(%286, 2)::Int64
│││││╻╷╷╷      axes   │    %299 = (Base.slt_int)(%297, 0)::Bool
││││││┃│││      map   │           (Base.ifelse)(%299, 0, %297)
│││││││╻╷╷       Type   │    %301 = (Base.slt_int)(%298, 0)::Bool
││││││││┃│        Type   │    %302 = (Base.ifelse)(%301, 0, %298)::Int64
││││││╻╷╷       checkbounds_indices   │    %303 = (Base.sle_int)(1, %170)::Bool
│││││││┃│        checkindex   │    %304 = (Base.sle_int)(%170, %302)::Bool
││││││││╻         &   │    %305 = (Base.and_int)(%303, %304)::Bool
│││││││╻         &   │    %306 = (Base.and_int)(%305, true)::Bool
││││││╻         &   │    %307 = (Base.an

Working with concrete types greatly sped up the computation, however, the C++ version is still 33x faster than Julia. Let's see what else can we optimize.

## Fix #2: Avoid List Comprehension Operations

The wonders of list comprehension may tempt you to do some line-efficient calculations, however, these will generally lead to a very inefficient computation. Take for example this list-comprehension sum:

In [18]:
fun1(n) = sum([i for i in 1:n])

@btime fun1(100);

  78.217 ns (1 allocation: 896 bytes)


Here is the version of the same function unrolled without the list comprehension:

In [19]:
function fun2(n)
    out = 0
    for i in 1:n
        out += i
    end
    return out
end

@btime fun2(100);

  1.303 ns (0 allocations: 0 bytes)


The difference is a 60x speed up.

In our function we had a Kronecker delta cross product that we were calculating in just one line as

```julia

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss -
                                  gsgm * const4/r^3 * 
                                  cross([i==j for i in 1:3], Pj.Gamma) ) #<----- HERE
      end
```

The alternative is to expand it into a few lines as

```julia

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss )
      end
            
      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]
```

The problem with list comprehension operations is that it has to allocate memory to build the array prior to operating. Just resist the temptation of using list comprehension to save yourself a few lines, and simply unroll it. As seen below we get a 1.5x speed up by unrolling this line:

In [20]:
function P2P_nocomprehension(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete}, 
                             g::Function, dgdr::Function)

  for Pi in targets 
    for Pj in sources    
            
      dX = Pi.X - Pj.X
      r = norm(dX) + EPS
            
      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)
            
      # K × Γp
      crss = cross(-const4 * dX / r^3, Pj.Gamma) 

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[:] += gsgm * crss

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss )
      end
            
      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_nocomprehension (generic function with 1 method)

In [21]:
prev_fun = this_fun
this_fun = P2P_nocomprehension
args = (particlesC, particlesC, g_wnklmns, dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_nocomprehension  is  1.53 times faster than   P2P_concretestruct (12.812ms vs 19.565ms)
C++                  is 21.92 times faster than  P2P_nocomprehension (0.584ms vs 12.812ms)


BenchmarkTools.Trial: 
  memory estimate:  37.43 MiB
  allocs estimate:  390625
  --------------
  minimum time:     12.920 ms (7.04% GC)
  median time:      14.482 ms (12.93% GC)
  mean time:        15.336 ms (13.32% GC)
  maximum time:     52.549 ms (72.10% GC)
  --------------
  samples:          326
  evals/sample:     1

## Fix #3: Reduce Allocation

Next, we notice that the benchmarking test is allotting an unusual amount of memory (37MiB) and allocation operation (390k). I am suspicious that this is an issue with Julia allowing arrays of dynamic sizes. The first step to solve this is to **do away with creating any internal variables of type arrays**. In the code bellow, notice that I had replaced the array variables `dX` and `crss` with float variables `dX1, dX2, dX3`, and `crss1, crss2, crss3`.

In [22]:
function P2P_noallocation(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g::Function, dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      dX1 = Pi.X[1] - Pj.X[1]
      dX2 = Pi.X[2] - Pj.X[2]
      dX3 = Pi.X[3] - Pj.X[3]
      r = norm(Pi.X - Pj.X) + EPS

      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)

      # K × Γp
      crss1, crss2, crss3 = -const4 / r^3 * cross(Pi.X - Pj.X, Pj.Gamma)

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[1] += gsgm * crss1
      Pi.U[2] += gsgm * crss2
      Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      Pi.J[1, 1] += aux * crss1 * dX1
      Pi.J[2, 1] += aux * crss2 * dX1
      Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      Pi.J[1, 2] += aux * crss1 * dX2
      Pi.J[2, 2] += aux * crss2 * dX2
      Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      Pi.J[1, 3] += aux * crss1 * dX3
      Pi.J[2, 3] += aux * crss2 * dX3
      Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end


P2P_noallocation (generic function with 1 method)

In [23]:
prev_fun = this_fun
this_fun = P2P_noallocation
args = (particlesC, particlesC, g_wnklmns, dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_noallocation     is  3.17 times faster than  P2P_nocomprehension (4.037ms vs 12.812ms)
C++                  is  6.91 times faster than     P2P_noallocation (0.584ms vs 4.037ms)


BenchmarkTools.Trial: 
  memory estimate:  7.39 MiB
  allocs estimate:  109375
  --------------
  minimum time:     4.023 ms (0.00% GC)
  median time:      4.400 ms (0.00% GC)
  mean time:        4.826 ms (9.85% GC)
  maximum time:     43.806 ms (86.73% GC)
  --------------
  samples:          1036
  evals/sample:     1

Here we have reduced the memory allocated from 37MiB to 7.4MiB, leading to a 3x speed up; however, the C++ version is still 7x faster. Let's see what else can we do to decrease that that memory allocation.

## Fix #4: No Functional Linear Algebra

The next thing to consider is that trying to do any **linear algebra operation (dot product, cross product, even norm) in a functional form (i.e., `dot(X,X)`, `cross(X,X)`, `norm(X,X)`) is more expensive that explicitely unfolding the operation into lines of code**. I am suspicious that this is a memory allocation problem since these functions need to allocate internal array variables to store computation and output the result. Here is the code without any functional linear algebra operations (notice that I no longer use `norm` nor `cross`):

In [24]:
function P2P_nolingalg(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g::Function, dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      dX1 = Pi.X[1] - Pj.X[1]
      dX2 = Pi.X[2] - Pj.X[2]
      dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)

      # K × Γp
      crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[1] += gsgm * crss1
      Pi.U[2] += gsgm * crss2
      Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      Pi.J[1, 1] += aux * crss1 * dX1
      Pi.J[2, 1] += aux * crss2 * dX1
      Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      Pi.J[1, 2] += aux * crss1 * dX2
      Pi.J[2, 2] += aux * crss2 * dX2
      Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      Pi.J[1, 3] += aux * crss1 * dX3
      Pi.J[2, 3] += aux * crss2 * dX3
      Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_nolingalg (generic function with 1 method)

In [25]:
prev_fun = this_fun
this_fun = P2P_nolingalg
args = (particlesC, particlesC, g_wnklmns, dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_nolingalg        is  1.94 times faster than     P2P_noallocation (2.085ms vs 4.037ms)
C++                  is  3.57 times faster than        P2P_nolingalg (0.584ms vs 2.085ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.087 ms (0.00% GC)
  median time:      2.163 ms (0.00% GC)
  mean time:        2.271 ms (0.00% GC)
  maximum time:     3.725 ms (0.00% GC)
  --------------
  samples:          2199
  evals/sample:     1

By doing away with linear algebra functions we have now done are not allocating any memory, reaching an extra 2x speed up.

## Fix #5: Fine Tuning

Notice that by now we are achieving times in the same order of magnitude than C++ (2.09ms in Julia vs 0.59ms in C++). What we did prior to this point were general principles that apply to any code that attempts to get high performance. What it follows now is fine tune our code in ways that only apply to the specific computation that we are performing.

For instance, recall that our P2P function receives any user-defined regularizing kernel that our function calls in through this lines:

```julia
function P2P(sources, targets, g::Function, dgdr::Function)

  for Pi in targets
    for Pj in sources
      .
      .
      .
      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)
      .
      .
      .
    end
  end
end
```

For the case of Winckelmans' kernel, `g` and `dgdr` look like this:
```julia
g_wnklmns(r) = r^3 * (r^2 + 2.5) / (r^2 + 1)^2.5
dgdr_wnklmns(r) = 7.5 * r^2 / (r^2 + 1)^3.5
```

We notice that each of these function calculate a power operation independently (`(r^2 + 1)^2.5` and `(r^2 + 1)^3.5`). I have observed that **any sort of non-integer power operation takes Julia more than than basic arithmetic operations or even space allocation**. We can save computation by merging this two functions and reusing the square root calculation as
```julia
function g_dgdr_wnklmns(r)
  aux0 = (r^2 + 1)^2.5
  
  # Returns g, dgdr
  return r^3*(r^2 + 2.5)/aux0, 7.5*r^2/(aux0*(r^2 + 1))
end
```

In [26]:
function g_dgdr_wnklmns(r)
  aux0 = (r^2 + 1)^2.5
  
  # Returns g, dgdr
  return r^3 * (r^2 + 2.5) / aux0, 7.5 * r^2 / (aux0*(r^2 + 1))
end

function P2P_reducesqrt(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      dX1 = Pi.X[1] - Pj.X[1]
      dX2 = Pi.X[2] - Pj.X[2]
      dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[1] += gsgm * crss1
      Pi.U[2] += gsgm * crss2
      Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      Pi.J[1, 1] += aux * crss1 * dX1
      Pi.J[2, 1] += aux * crss2 * dX1
      Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      Pi.J[1, 2] += aux * crss1 * dX2
      Pi.J[2, 2] += aux * crss2 * dX2
      Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      Pi.J[1, 3] += aux * crss1 * dX3
      Pi.J[2, 3] += aux * crss2 * dX3
      Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_reducesqrt (generic function with 1 method)

In [27]:
prev_fun = P2P_nolingalg
this_fun = P2P_reducesqrt
args = (particlesC, particlesC, g_dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_reducesqrt       is  1.54 times faster than        P2P_nolingalg (1.350ms vs 2.085ms)
C++                  is  2.31 times faster than       P2P_reducesqrt (0.584ms vs 1.350ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.349 ms (0.00% GC)
  median time:      1.392 ms (0.00% GC)
  mean time:        1.433 ms (0.00% GC)
  maximum time:     2.192 ms (0.00% GC)
  --------------
  samples:          3486
  evals/sample:     1

Hence, by **simply avoiding one extra power calculation we have now gained a 1.5x speed up.**

## Other Tips

### @inbounds

https://docs.julialang.org/en/v1/devdocs/boundscheck/index.html
>Like many modern programming languages, Julia uses bounds checking to ensure program safety when accessing arrays. In tight inner loops or other performance critical situations, you may wish to skip these bounds checks to improve runtime performance. For instance, in order to emit vectorized (SIMD) instructions, your loop body cannot contain branches, and thus cannot contain bounds checks. Consequently, Julia includes an @inbounds(...) macro to tell the compiler to skip such bounds checks within the given block. User-defined array types can use the @boundscheck(...) macro to achieve context-sensitive code selection.

In [28]:
? @inbounds

```
@inbounds(blk)
```

Eliminates array bounds checking within expressions.

In the example below the in-range check for referencing element `i` of array `A` is skipped to improve performance.

```julia
function sum(A::AbstractArray)
    r = zero(eltype(A))
    for i = 1:length(A)
        @inbounds r += A[i]
    end
    return r
end
```

!!! warning
    Using `@inbounds` may return incorrect results/crashes/corruption for out-of-bounds indices. The user is responsible for checking it manually. Only use `@inbounds` when it is certain from the information locally available that all accesses are in bounds.



In [29]:
function P2P_inbounds(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      @inbounds dX1 = Pi.X[1] - Pj.X[1]
      @inbounds dX2 = Pi.X[2] - Pj.X[2]
      @inbounds dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      @inbounds crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      @inbounds crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      @inbounds crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      @inbounds Pi.U[1] += gsgm * crss1
      @inbounds Pi.U[2] += gsgm * crss2
      @inbounds Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      @inbounds Pi.J[1, 1] += aux * crss1 * dX1
      @inbounds Pi.J[2, 1] += aux * crss2 * dX1
      @inbounds Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      @inbounds Pi.J[1, 2] += aux * crss1 * dX2
      @inbounds Pi.J[2, 2] += aux * crss2 * dX2
      @inbounds Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      @inbounds Pi.J[1, 3] += aux * crss1 * dX3
      @inbounds Pi.J[2, 3] += aux * crss2 * dX3
      @inbounds Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      @inbounds Pi.J[2, 1] -= aux * Pj.Gamma[3]
      @inbounds Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      @inbounds Pi.J[1, 2] += aux * Pj.Gamma[3]
      @inbounds Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      @inbounds Pi.J[1, 3] -= aux * Pj.Gamma[2]
      @inbounds Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_inbounds (generic function with 1 method)

In [30]:
prev_fun = P2P_reducesqrt
this_fun = P2P_inbounds
args = (particlesC, particlesC, g_dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_inbounds         is  1.01 times faster than       P2P_reducesqrt (1.338ms vs 1.350ms)
C++                  is  2.29 times faster than         P2P_inbounds (0.584ms vs 1.338ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.338 ms (0.00% GC)
  median time:      1.380 ms (0.00% GC)
  mean time:        1.437 ms (0.00% GC)
  maximum time:     2.404 ms (0.00% GC)
  --------------
  samples:          3475
  evals/sample:     1

In our case, `@inbounds` does not seem to help at all.

### @simd

https://docs.julialang.org/en/v1/manual/performance-tips/index.html#man-performance-annotations-1
> Write @simd in front of for loops to promise that the iterations are independent and may be reordered. Note that in many cases, Julia can automatically vectorize code without the @simd macro; it is only beneficial in cases where such a transformation would otherwise be illegal, including cases like allowing floating-point re-associativity and ignoring dependent memory accesses (@simd ivdep). Again, be very careful when asserting @simd as erroneously annotating a loop with dependent iterations may result in unexpected results. In particular, note that setindex! on some AbstractArray subtypes is inherently dependent upon iteration order. This feature is experimental and could change or disappear in future versions of Julia.

In [31]:
? @simd

```
@simd
```

Annotate a `for` loop to allow the compiler to take extra liberties to allow loop re-ordering

!!! warning
    This feature is experimental and could change or disappear in future versions of Julia. Incorrect use of the `@simd` macro may cause unexpected results.


The object iterated over in a `@simd for` loop should be a one-dimensional range. By using `@simd`, you are asserting several properties of the loop:

```
* It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
* Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.
```

In many cases, Julia is able to automatically vectorize inner for loops without the use of `@simd`. Using `@simd` gives the compiler a little extra leeway to make it possible in more situations. In either case, your inner loop should have the following properties to allow vectorization:

```
* The loop must be an innermost loop
* The loop body must be straight-line code. Therefore, [`@inbounds`](@ref) is
  currently needed for all array accesses. The compiler can sometimes turn
  short `&&`, `||`, and `?:` expressions into straight-line code if it is safe
  to evaluate all operands unconditionally. Consider using the [`ifelse`](@ref)
  function instead of `?:` in the loop if it is safe to do so.
* Accesses must have a stride pattern and cannot be "gathers" (random-index
  reads) or "scatters" (random-index writes).
* The stride should be unit stride.
```

!!! note
    The `@simd` does not assert by default that the loop is completely free of loop-carried memory dependencies, which is an assumption that can easily be violated in generic code. If you are writing non-generic code, you can use `@simd ivdep for ... end` to also assert that:

    ```
    * There exists no loop-carried memory dependencies
    * No iteration ever waits on a previous iteration to make forward progress.
    ```



In [32]:
function P2P_simd(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  for Pi in targets
    @simd for Pj in sources
      
      @inbounds dX1 = Pi.X[1] - Pj.X[1]
      @inbounds dX2 = Pi.X[2] - Pj.X[2]
      @inbounds dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      @inbounds crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      @inbounds crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      @inbounds crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      @inbounds Pi.U[1] += gsgm * crss1
      @inbounds Pi.U[2] += gsgm * crss2
      @inbounds Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      @inbounds Pi.J[1, 1] += aux * crss1 * dX1
      @inbounds Pi.J[2, 1] += aux * crss2 * dX1
      @inbounds Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      @inbounds Pi.J[1, 2] += aux * crss1 * dX2
      @inbounds Pi.J[2, 2] += aux * crss2 * dX2
      @inbounds Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      @inbounds Pi.J[1, 3] += aux * crss1 * dX3
      @inbounds Pi.J[2, 3] += aux * crss2 * dX3
      @inbounds Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      @inbounds Pi.J[2, 1] -= aux * Pj.Gamma[3]
      @inbounds Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      @inbounds Pi.J[1, 2] += aux * Pj.Gamma[3]
      @inbounds Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      @inbounds Pi.J[1, 3] -= aux * Pj.Gamma[2]
      @inbounds Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_simd (generic function with 1 method)

In [33]:
prev_fun = P2P_reducesqrt
this_fun = P2P_simd
args = (particlesC, particlesC, g_dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_simd             is  1.01 times faster than       P2P_reducesqrt (1.341ms vs 1.350ms)
C++                  is  2.29 times faster than             P2P_simd (0.584ms vs 1.341ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.340 ms (0.00% GC)
  median time:      1.425 ms (0.00% GC)
  mean time:        1.510 ms (0.00% GC)
  maximum time:     2.972 ms (0.00% GC)
  --------------
  samples:          3303
  evals/sample:     1

In [34]:
@noinline function P2P_simd2(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  @inbounds for Pi in targets
    @inbounds @simd for Pj in sources
      
      @inbounds dX1 = Pi.X[1] - Pj.X[1]
      @inbounds dX2 = Pi.X[2] - Pj.X[2]
      @inbounds dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      @inbounds crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      @inbounds crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      @inbounds crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      @inbounds Pi.U[1] += gsgm * crss1
      @inbounds Pi.U[2] += gsgm * crss2
      @inbounds Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      @inbounds Pi.J[1, 1] += aux * crss1 * dX1
      @inbounds Pi.J[2, 1] += aux * crss2 * dX1
      @inbounds Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      @inbounds Pi.J[1, 2] += aux * crss1 * dX2
      @inbounds Pi.J[2, 2] += aux * crss2 * dX2
      @inbounds Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      @inbounds Pi.J[1, 3] += aux * crss1 * dX3
      @inbounds Pi.J[2, 3] += aux * crss2 * dX3
      @inbounds Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      @inbounds Pi.J[2, 1] -= aux * Pj.Gamma[3]
      @inbounds Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      @inbounds Pi.J[1, 2] += aux * Pj.Gamma[3]
      @inbounds Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      @inbounds Pi.J[1, 3] -= aux * Pj.Gamma[2]
      @inbounds Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_simd2 (generic function with 1 method)

In [35]:
prev_fun = P2P_reducesqrt
this_fun = P2P_simd2
args = (particlesC, particlesC, g_dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_simd2            is  1.01 times faster than       P2P_reducesqrt (1.341ms vs 1.350ms)
C++                  is  2.29 times faster than            P2P_simd2 (0.584ms vs 1.341ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.340 ms (0.00% GC)
  median time:      1.387 ms (0.00% GC)
  mean time:        1.450 ms (0.00% GC)
  maximum time:     2.461 ms (0.00% GC)
  --------------
  samples:          3443
  evals/sample:     1

Nop, it doesn't help at all.

### @fastmath

https://docs.julialang.org/en/v1/manual/performance-tips/index.html#man-performance-annotations-1
> Use @fastmath to allow floating point optimizations that are correct for real numbers, but lead to differences for IEEE numbers. Be careful when doing this, as this may change numerical results. This corresponds to the -ffast-math option of clang.

In [36]:
function P2P_fastmath(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      @fastmath dX1 = Pi.X[1] - Pj.X[1]
      @fastmath dX2 = Pi.X[2] - Pj.X[2]
      @fastmath dX3 = Pi.X[3] - Pj.X[3]
      @fastmath r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      @fastmath crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      @fastmath crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      @fastmath crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      @fastmath Pi.U[1] += gsgm * crss1
      @fastmath Pi.U[2] += gsgm * crss2
      @fastmath Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      @fastmath aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      @fastmath Pi.J[1, 1] += aux * crss1 * dX1
      @fastmath Pi.J[2, 1] += aux * crss2 * dX1
      @fastmath Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      @fastmath Pi.J[1, 2] += aux * crss1 * dX2
      @fastmath Pi.J[2, 2] += aux * crss2 * dX2
      @fastmath Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      @fastmath Pi.J[1, 3] += aux * crss1 * dX3
      @fastmath Pi.J[2, 3] += aux * crss2 * dX3
      @fastmath Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      @fastmath aux = - const4 * gsgm / r^3
      # j=1
      @fastmath Pi.J[2, 1] -= aux * Pj.Gamma[3]
      @fastmath Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      @fastmath Pi.J[1, 2] += aux * Pj.Gamma[3]
      @fastmath Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      @fastmath Pi.J[1, 3] -= aux * Pj.Gamma[2]
      @fastmath Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_fastmath (generic function with 1 method)

In [37]:
prev_fun = P2P_reducesqrt
this_fun = P2P_fastmath
args = (particlesC, particlesC, g_dgdr_wnklmns)

compare(this_fun, prev_fun, args; reverse=true)
printcomparison(this_fun, "C++", false)

@benchmark this_fun(args...)

P2P_fastmath         is  0.97 times faster than       P2P_reducesqrt (1.396ms vs 1.350ms)
C++                  is  2.39 times faster than         P2P_fastmath (0.584ms vs 1.396ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.397 ms (0.00% GC)
  median time:      1.441 ms (0.00% GC)
  mean time:        1.503 ms (0.00% GC)
  maximum time:     2.707 ms (0.00% GC)
  --------------
  samples:          3322
  evals/sample:     1

Did it actually make it slower? 

### @code_warntype

I already mentioned this, but here I'm just double checking that everything is compiling fine

In [38]:
@code_warntype P2P_reducesqrt(particlesC, particlesC, g_dgdr_wnklmns)

Body::Nothing
│╻╷╷  iterate12 1 ── %1   = (Base.arraylen)(targets)::Int64
││╻╷   iterate   │    %2   = (Base.sle_int)(0, %1)::Bool
│││╻    <   │    %3   = (Base.bitcast)(UInt64, %1)::UInt64
││││╻    <   │    %4   = (Base.ult_int)(0x0000000000000000, %3)::Bool
││││╻    &   │    %5   = (Base.and_int)(%2, %4)::Bool
│││     └───        goto #3 if not %5
│││╻    getindex   2 ── %7   = (Base.arrayref)(false, targets, 1)::ParticleConcrete
│││     └───        goto #4
│││     3 ──        goto #4
││      4 ┄─ %10  = φ (#2 => false, #3 => true)::Bool
││      │    %11  = φ (#2 => %7)::ParticleConcrete
││      │    %12  = φ (#2 => 2)::Int64
││      └───        goto #5
│       5 ── %14  = (Base.not_int)(%10)::Bool
│       └───        goto #24 if not %14
│       6 ┄─ %16  = φ (#5 => %11, #23 => %273)::ParticleConcrete
│       │    %17  = φ (#5 => %12, #23 => %274)::Int64
│╻╷╷  iterate13 │    %18  = (Base.arraylen)(sources)::Int64
││╻╷   iterate   │    %19  = (Base.sle_int)(0, %18)::Bool
│││╻    <   │

│╻    *   │    %98  = (Base.mul_float)(%44, %97)::Float64
│╻    -   │    %99  = (Base.sub_float)(%95, %98)::Float64
│╻    *   │    %100 = (Base.mul_float)(%92, %99)::Float64
│╻    getproperty29 │    %101 = (Base.getfield)(%16, :U)::Array{Float64,1}
│╻    getindex   │    %102 = (Base.arrayref)(true, %101, 1)::Float64
│╻    *   │    %103 = (Base.mul_float)(%66, %78)::Float64
│╻    +   │    %104 = (Base.add_float)(%102, %103)::Float64
│╻    getproperty   │    %105 = (Base.getfield)(%16, :U)::Array{Float64,1}
│╻    setindex!   │           (Base.arrayset)(true, %105, %104, 1)
│╻    getproperty30 │    %107 = (Base.getfield)(%16, :U)::Array{Float64,1}
│╻    getindex   │    %108 = (Base.arrayref)(true, %107, 2)::Float64
│╻    *   │    %109 = (Base.mul_float)(%66, %89)::Float64
│╻    +   │    %110 = (Base.add_float)(%108, %109)::Float64
│╻    getproperty   │    %111 = (Base.getfield)(%16, :U)::Array{Float64,1}
│╻    setindex!   │           (Base.arrayset)(true, %111, %110, 2)
│╻    getproperty3

││╻╷   *   │    %190 = (Base.mul_float)(%62, %62)::Float64
│││┃    *   │    %191 = (Base.mul_float)(%190, %62)::Float64
│╻    /   │    %192 = (Base.div_float)(%189, %191)::Float64
│╻    getproperty51 │    %193 = (Base.getfield)(%16, :J)::Array{Float64,2}
│╻    getindex   │    %194 = (Base.arrayref)(true, %193, 2, 1)::Float64
│╻    getproperty   │    %195 = (Base.getfield)(%33, :Gamma)::Array{Float64,1}
│╻    getindex   │    %196 = (Base.arrayref)(true, %195, 3)::Float64
│╻    *   │    %197 = (Base.mul_float)(%192, %196)::Float64
│╻    -   │    %198 = (Base.sub_float)(%194, %197)::Float64
│╻    getproperty   │    %199 = (Base.getfield)(%16, :J)::Array{Float64,2}
│╻    setindex!   │           (Base.arrayset)(true, %199, %198, 2, 1)
│╻    getproperty52 │    %201 = (Base.getfield)(%16, :J)::Array{Float64,2}
│╻    getindex   │    %202 = (Base.arrayref)(true, %201, 3, 1)::Float64
│╻    getproperty   │    %203 = (Base.getfield)(%33, :Gamma)::Array{Float64,1}
│╻    getindex   │    %204 = (Base

### @trace

https://github.com/MikeInnes/Traceur.jl
> Traceur is essentially a codified version of the Julia performance tips. You run your code, it tells you about any obvious performance traps.

In [39]:
using Traceur

# @trace P2P_reducesqrt(particlesC, particlesC, g_dgdr_wnklmns)
# @trace P2P_FINAL(particlesC, particlesC, g_dgdr_wnklmns)

I have tried running this before, but it never finishes, hence I decided to comented it out.

## FINAL

In [40]:
function g_dgdr_wnklmns(r)
  aux0 = (r^2 + 1)^2.5
  
  # Returns g, dgdr
  return r^3 * (r^2 + 2.5) / aux0, 7.5 * r^2 / (aux0*(r^2 + 1))
end

function P2P_FINAL(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      dX1 = Pi.X[1] - Pj.X[1]
      dX2 = Pi.X[2] - Pj.X[2]
      dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[1] += gsgm * crss1
      Pi.U[2] += gsgm * crss2
      Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      Pi.J[1, 1] += aux * crss1 * dX1
      Pi.J[2, 1] += aux * crss2 * dX1
      Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      Pi.J[1, 2] += aux * crss1 * dX2
      Pi.J[2, 2] += aux * crss2 * dX2
      Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      Pi.J[1, 3] += aux * crss1 * dX3
      Pi.J[2, 3] += aux * crss2 * dX3
      Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end

P2P_FINAL (generic function with 1 method)

In [43]:
this_fun = P2P_FINAL
args = (particlesC, particlesC, g_dgdr_wnklmns)

compare(this_fun, P2P_general, args; reverse=true)
compare(this_fun, "C++", args)

@benchmark this_fun(args...)

P2P_FINAL            is 49.88 times faster than          P2P_general (1.349ms vs 67.297ms)
C++                  is  2.31 times faster than            P2P_FINAL (0.584ms vs 1.349ms)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.349 ms (0.00% GC)
  median time:      1.527 ms (0.00% GC)
  mean time:        1.600 ms (0.00% GC)
  maximum time:     2.744 ms (0.00% GC)
  --------------
  samples:          3114
  evals/sample:     1

In conclusion, after all the speed up, C++ is still 2.3x faster than Julia; but be not discouraged, we are talking about a dynamic language that is **only 0.4x slower than C++**! =]

**NOTE:** In a subsequent test I commented out the two power operations (`r = sqrt(...)` and `gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)`) and the resulting wall-clock time went down from 1.35ms to 0.185ms, meaning that at this stage the overhead is on non-algebraic operations and that the function could further be optimized by finding a more efficient way of calculation powers in Julia.

## Discussion

Finally, let's take a second to compare where we started and where we ended. Our initial Julia function was very compact and understandable, however it was more than 100x slower than its C++ version:

```julia
g_wnklmns(r) = r^3 * (r^2 + 2.5) / (r^2 + 1)^2.5
dgdr_wnklmns(r) = 7.5 * r^2 / (r^2 + 1)^3.5

function P2P_general(sources::Array{ParticleAmbiguous},
                     targets::Array{ParticleAmbiguous}, 
                     g::Function, dgdr::Function)

  for Pi in targets 
    for Pj in sources    
            
      dX = Pi.X - Pj.X
      r = norm(dX) + EPS
            
      # Regularizing function and deriv
      gsgm = g(r/Pj.sigma)
      dgsgmdr = dgdr(r/Pj.sigma)  
            
      # K × Γp
      crss = cross(-const4 * dX / r^3, Pj.Gamma) 

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[:] += gsgm * crss

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      for j in 1:3
        Pi.J[:, j] += ( dX[j]/(Pj.sigma*r)*dgsgmdr * crss -
                                  gsgm * 3*dX[j]/r^2*crss -
                                  gsgm * const4/r^3 * 
                                  cross([i==j for i in 1:3], Pj.Gamma) )
      end

    end
  end
end
```
```
C++                  is 115.16 times faster than          P2P_general (0.584ms vs 67.297ms)
Out[9]:
BenchmarkTools.Trial: 
  memory estimate:  69.86 MiB
  allocs estimate:  1437500
  --------------
  minimum time:     66.281 ms (4.12% GC)
  median time:      69.037 ms (4.52% GC)
  mean time:        71.489 ms (5.60% GC)
  maximum time:     126.785 ms (37.55% GC)
  --------------
  samples:          70
  evals/sample:     1
```

After all the optimization we end up with about twice the amount of lines, but 50x times faster that the original version and only 2.3x slower than C++:

```julia
function g_dgdr_wnklmns(r)
  aux0 = (r^2 + 1)^2.5
  
  # Returns g, dgdr
  return r^3 * (r^2 + 2.5) / aux0, 7.5 * r^2 / (aux0*(r^2 + 1))
end

function P2P_FINAL(sources::Array{ParticleConcrete},
                             targets::Array{ParticleConcrete},
                             g_dgdr::Function)

  for Pi in targets
    for Pj in sources
      
      dX1 = Pi.X[1] - Pj.X[1]
      dX2 = Pi.X[2] - Pj.X[2]
      dX3 = Pi.X[3] - Pj.X[3]
      r = sqrt(dX1*dX1 + dX2*dX2 + dX3*dX3) + EPS

      # Regularizing function and deriv
      gsgm, dgsgmdr = g_dgdr(r/Pj.sigma)

      # K × Γp
      crss1 = -const4 / r^3 * ( dX2*Pj.Gamma[3] - dX3*Pj.Gamma[2] )
      crss2 = -const4 / r^3 * ( dX3*Pj.Gamma[1] - dX1*Pj.Gamma[3] )
      crss3 = -const4 / r^3 * ( dX1*Pj.Gamma[2] - dX2*Pj.Gamma[1] )

      # U = ∑g_σ(x-xp) * K(x-xp) × Γp
      Pi.U[1] += gsgm * crss1
      Pi.U[2] += gsgm * crss2
      Pi.U[3] += gsgm * crss3

      # ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = dgsgmdr/(Pj.sigma*r)* - 3*gsgm /r^2
      # j=1
      Pi.J[1, 1] += aux * crss1 * dX1
      Pi.J[2, 1] += aux * crss2 * dX1
      Pi.J[3, 1] += aux * crss3 * dX1
      # j=2
      Pi.J[1, 2] += aux * crss1 * dX2
      Pi.J[2, 2] += aux * crss2 * dX2
      Pi.J[3, 2] += aux * crss3 * dX2
      # j=3
      Pi.J[1, 3] += aux * crss1 * dX3
      Pi.J[2, 3] += aux * crss2 * dX3
      Pi.J[3, 3] += aux * crss3 * dX3

      # Adds the Kronecker delta term
      aux = - const4 * gsgm / r^3
      # j=1
      Pi.J[2, 1] -= aux * Pj.Gamma[3]
      Pi.J[3, 1] += aux * Pj.Gamma[2]
      # j=2
      Pi.J[1, 2] += aux * Pj.Gamma[3]
      Pi.J[3, 2] -= aux * Pj.Gamma[1]
      # j=3
      Pi.J[1, 3] -= aux * Pj.Gamma[2]
      Pi.J[2, 3] += aux * Pj.Gamma[1]

    end
  end
end
```
```
P2P_FINAL            is 49.88 times faster than          P2P_general (1.349ms vs 67.297ms)
C++                  is  2.31 times faster than            P2P_FINAL (0.584ms vs 1.349ms)
Out[43]:
BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.349 ms (0.00% GC)
  median time:      1.527 ms (0.00% GC)
  mean time:        1.600 ms (0.00% GC)
  maximum time:     2.744 ms (0.00% GC)
  --------------
  samples:          3114
  evals/sample:     1
```

Oddly enough, through the optimization process we naturally morphed the code into something that looks very similar to the C++ version:

```c++
void P2P(Cell * Ci, Cell * Cj) {
  real_t const4 = 1/(4*M_PI);
  Particle * Pi = Ci->particle;
  Particle * Pj = Cj->particle;

  for (int i=0; i<Ci->numParticles; i++) {
    vec3 U = 0;
    vec9 J = 0;
    
    for (int j=0; j<Cj->numParticles; j++) {
      vec3 dX = Pi[i].X - Pj[j].X;
      real_t r = std::pow(dX[0]*dX[0] + dX[1]*dX[1] + dX[2]*dX[2], 0.5) + EPS;
      real_t ros = r/Pj[j].sigma;

      // u(x) = ∑g_σ(x−xp) K(x−xp) × Γp
      real_t aux0 = std::pow(ros*ros + 1.0, 2.5);
      real_t g_sgm = ros*ros*ros * (ros*ros + 2.5) / aux0;
      real_t dgdr = 7.5 * ros*ros / ( aux0 * (ros*ros + 1.0) );
      real_t aux = (- const4 / r*r*r) * g_sgm;
      U[0] += ( dX[1]*Pj[j].q[2] - dX[2]*Pj[j].q[1] ) * aux;
      U[1] += ( dX[2]*Pj[j].q[0] - dX[0]*Pj[j].q[2] ) * aux;
      U[2] += ( dX[0]*Pj[j].q[1] - dX[1]*Pj[j].q[0] ) * aux;

      // ∂u∂xj(x) = ∑[ ∂gσ∂xj(x−xp) * K(x−xp)×Γp + gσ(x−xp) * ∂K∂xj(x−xp)×Γp ]
      aux = (- const4 / r*r*r) * (dgdr/(Pj[j].sigma*r) - 3.0*g_sgm/r*r);
      for (int k=0; k<3; k++){
        J[0 + k] += ( dX[1]*Pj[j].q[2] - dX[2]*Pj[j].q[1] )* aux*dX[k];
        J[3 + k] += ( dX[2]*Pj[j].q[0] - dX[0]*Pj[j].q[2] )* aux*dX[k];
        J[6 + k] += ( dX[0]*Pj[j].q[1] - dX[1]*Pj[j].q[0] )* aux*dX[k];
      }

      // Adds the Kronecker delta term
      aux = - const4 * g_sgm / r*r*r;
      // k=1
      J[3 + 0] -= aux * Pj[j].q[2];
      J[6 + 0] += aux * Pj[j].q[1];
      // k=2
      J[0 + 1] += aux * Pj[j].q[2];
      J[6 + 1] -= aux * Pj[j].q[0];
      // k=3
      J[0 + 2] -= aux * Pj[j].q[1];
      J[3 + 2] += aux * Pj[j].q[0];
    }

    Pi[i].p += U;
    Pi[i].J += J;
  }
}
```

Hence, If you are trying to get a computation-efficient piece of code, my recommendation is to **forget about elegant and line-efficient coding ("pythonic" some would say), and code in C++-esque style from the beginning**.

## Conclusions

* Without foreknowledge of the types to be handled in an operation, Julia can't optimize the function during compilation. There is no problem with ambiguous types in function definitions since the JIT will compile a well-defined version of the function based on the arguments that is given on the fly, but this is not the case for structs. **Always define your structs with its [properties explicitely specified as concrete types](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Type-declarations-1)**.


* [`@code_warntype`](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#man-code-warntype-1) is your best friend when trying to optimize your computation.


* The first thing to look at after making sure that all your types are well defined as concrete types, is **memory allocation**. If your function is allocating an insane amount of memory, that's an indication that you are saving yourself some lines of code in sacrifice of doing some efficient computation (*e.g.,* inline list-comprehension operations or array algebra instead of unrolling the operations into explicit code). I recommend using [`@time`](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Measure-performance-with-[@time](@ref)-and-pay-attention-to-memory-allocation-1) or [`@benchmark` (from BenchmarkTools.jl)](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md) to check memory allocation. 


* **Avoid storing intermediate calculations in any kind of internal arrays, just use Float variables**. For some reason Julia spends a lot of time trying to allocate space for internal arrays.


* Any **non-integer power operations (`^` or `sqrt`) are very expensive** in Julia. If possible, reduce these operations by storing in memory the calculations and reusing them.


* At all cost, **avoid using LinearAlgebra functions** (like `dot`, `cross`, `norm`) as they will require allocating memory for internal arrays.


* Use [`@inbounds`](https://docs.julialang.org/en/v1/base/base/#Base.@inbounds) whenever accessing/writting an array of which you know its dimensions a priori.


* [`@simd`](https://docs.julialang.org/en/v1/base/base/#Base.SimdLoop.@simd) and [`@fastmath`](https://docs.julialang.org/en/v1/base/math/#Base.FastMath.@fastmath) may actually slow down the performance when not needed.


* If you are trying to get a computation-efficient piece of code, my recommendation is to **forget about elegant and line-efficient coding ("pythonic" some would say), and code in C++-esque style from the beginning**.